In [51]:
class TweetExtractor():

    def __init__(self, APP_KEY, APP_SECRET):
        from twython import Twython
        self.twitter = Twython(APP_KEY, APP_SECRET)

    def getName(self):
        self.username = input("Enter screen_name?")

    def getTimeline(self,screen_name,count, max_id=None):
        if(max_id == None):
            return self.twitter.get_user_timeline(
                screen_name=screen_name, count=count)
        else:
            return self.twitter.get_user_timeline(
                screen_name=screen_name, count=count, max_id=max_id)

    def getLastID(self, user_timeline):
        return user_timeline[0]['id']-1

    def getAllTweets(self,screen_name):
        self.user_timeline = self.getTimeline(screen_name, 1)
        for _i in range(16):
            new200tweets = self.getTimeline(screen_name, 200, self.getLastID(self.user_timeline))
            self.user_timeline.extend(new200tweets)
        return self.user_timeline
    
    def extractTweetsText(self,user_timeline):
        return [item["text"] for item in user_timeline]


    def extractTweetsTextWithRef(self, user_timeline):
        return [{"text":item["text"],"id_str" : item["id_str"]} for item in user_timeline]



    def writeToFile(self,user_timeline):
        import codecs
        file = codecs.open("output.txt", "w", "utf-8")
        file.write(user_timeline)
        file.close()


In [94]:
from Config import *
tweetExtractor = TweetExtractor(APP_KEY, APP_SECRET)
tweets_list = tweetExtractor.extractTweetsTextWithRef(tweets)


def getAllUsrTweets():
    username = input("What username?")
    tweets = tweetExtractor.getAllTweets(username)
    return tweets


tweets = getAllUsrTweets()


What username?mahdi_raf


In [97]:
print(len(tweets))
print((tweets[0]).keys())

3201
dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'extended_entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])


In [46]:
sensitiveWords = ["جمهوری", "ج.ا", "اسلامی", "ایران", "مرگ", "جاعش", "خامنه", "احمدی نژاد",
                  "روحانی", "@azarijahromi", "اینا", "مجلس", "قوه قضاییه", "رئیس", "آخوند", "شاه ", "حجاب", "آمریکا"]

In [88]:
import codecs
file = codecs.open("output.html", "w", "utf-8")
file.write("<div style='direction: rtl;'>")
for i in range(len(tweets_list)):
    foundWords = []
    for j in range(len(sensitiveWords)):
        if(len(sensitiveWords[j]) == 0):
            continue
        if tweets_list[i]["text"].find(sensitiveWords[j]) > -1:
            foundWords.append(sensitiveWords[j])
    if len(foundWords)>0:
        file.write("<p><a target='_blank' href='https://twitter.com/i/web/status/" + tweets_list[i]["id_str"] +"'>"+tweets_list[i]["text"]+"</a>")
        file.write("       "+' '.join([str(elem) for elem in foundWords])+"</p>")
        file.write("<br>")

file.write("</div>")
file.close()